In [1]:
import numpy as np

In [2]:
def random_initialization(A,rank):
    number_of_documents = A.shape[0]
    number_of_terms = A.shape[1]
    W = np.random.uniform(1,2,(number_of_documents,rank))
    H = np.random.uniform(1,2,(rank,number_of_terms))
    return W,H
                          

def nndsvd_initialization(A,rank):

    u,s,v=np.linalg.svd(A,full_matrices=False)

    v=v.T
    w=np.zeros((A.shape[0],rank))
    h=np.zeros((rank,A.shape[1]))

    w[:,0]=np.sqrt(s[0])*np.abs(u[:,0])
    h[0,:]=np.sqrt(s[0])*np.abs(v[:,0].T)

    for i in range(1,rank):
        
        ui=u[:,i]
        vi=v[:,i]
        ui_pos=(ui>=0)*ui
        ui_neg=(ui<0)*-ui
        vi_pos=(vi>=0)*vi
        vi_neg=(vi<0)*-vi
        
        ui_pos_norm=np.linalg.norm(ui_pos,2)
        ui_neg_norm=np.linalg.norm(ui_neg,2)
        vi_pos_norm=np.linalg.norm(vi_pos,2)
        vi_neg_norm=np.linalg.norm(vi_neg,2)
        
        norm_pos=ui_pos_norm*vi_pos_norm
        norm_neg=ui_neg_norm*vi_neg_norm
        
        if norm_pos>=norm_neg:
            w[:,i]=np.sqrt(s[i]*norm_pos)/ui_pos_norm*ui_pos
            h[i,:]=np.sqrt(s[i]*norm_pos)/vi_pos_norm*vi_pos.T
        else:
            w[:,i]=np.sqrt(s[i]*norm_neg)/ui_neg_norm*ui_neg
            h[i,:]=np.sqrt(s[i]*norm_neg)/vi_neg_norm*vi_neg.T

    return w,h

In [ ]:
# def mur_optimization_RNMF( X, k, max_iteration, init_mode):

#     #X_c = the X with clean data
#     #X is the polluted data


#     #U.shape is m * k,  V.shape is k * n
#     #m = numbers of features/ elements of 1 column vector of X.
#     #n is the numbers of training examples

#     #TODO

#     #implement earlier stop


#     #X should be 

#     if init_mode == 'random':
#         U ,V = random_initialization(X_c,k)    
#     elif init_mode == 'nndsvd':
#         U ,V = nndsvd_initialization(X_c,k) 

    
#     error = []

#     e = 1.0e-10

#     for iter in range(max_iteration):

#         #update U






    

In [3]:
def mur_optimization_nmf_stock(X, k, max_iter, init_mode):

    if init_mode == 'random':
        U ,V = random_initialization(X, k)
    elif init_mode == 'nndsvd':
        U ,V = nndsvd_initialization(X, k) 


    loss_iter = []


    for iter in range(max_iter):

        #update V
        U_tX = U.T @ X
        U_tUV = U.T @ U @ V

        for i in range(np.size(V, 0)): #i row
            for j in range(np.size(V, 1)): #j column
                V[i,j] = V[i,j] * U_tX[i,j] / U_tUV[i,j]


        #update U
        XV_t = X @ V.T
        UVV_t = U @ V @ V.T

        for i in range(np.size(U, 0)): #i row
            for j in range(np.size(U, 1)): #j column
                U[i,j] = U[i,j] * XV_t[i,j] / UVV_t[i,j]

        
        loss = np.linalg.norm(X - (U @ V), 'fro' )  #Frobenius norm, by default
        loss_iter.append(loss)
    

    return U, V, loss_iter




In [16]:
def mur_optimization_robust_nmf_l1(X, k, reg_lambda, max_iter, init_mode):
    
    if init_mode == 'random':
        U ,V = random_initialization(X, k)
    elif init_mode == 'nndsvd':
        U ,V = nndsvd_initialization(X, k) 


    #generate E

    #method 1
    # E is initialized with all zeros
    #E = np.zeros(X.shape)

    #method 2
    # E = X - UV
    E = X - (U @ V)



    #E = np.random.random(())


    
    loss_iter = []

    
    for iter in range(max_iter):


        # update U

        #get X^
        X_hat = np.subtract(X,E)

        #check negative constrain  X - E >= 0
        if X_hat.sum() <= 0:
            print("Error, noise is greater than X")


        X_hat_V_t = X_hat @ V.T
        UVV_t = U @ V @ V.T

        for i in range(np.size(U, 0)): #i row
            for j in range(np.size(U,1)): #j column
                U[i,j] = U[i,j] * X_hat_V_t[i,j] / UVV_t[i,j]
        

        # update V, E simultaneously

        E_p = (np.absolute(E) + E) / 2

        E_n = (np.absolute(E) - E) / 2


        #compute V dash
        V_dash = np.concatenate((V, E_p, E_n), axis=0)


        #compute X dash
        X_dash = np.concatenate( ( X, np.zeros((1, X.shape[1])) ), axis=0)


        #compute U dash
        U_1 = np.concatenate( ( U, np.zeros((1,U.shape[1])) ), axis=0)

        reg_parameter = np.sqrt(reg_lambda) * np.exp(1)

        arr_below_I = np.full( (1, U.shape[0]),  reg_parameter)

        I_U = np.concatenate( (np.identity(U.shape[0]), arr_below_I ), axis=0 )

        N_I_U = np.concatenate( (  (-1)*(np.identity(U.shape[0])) , arr_below_I ), axis=0 )

        U_dash = np.concatenate( (U_1, I_U, N_I_U), axis=1  )
        

        #compute S
        S = np.absolute( U_dash.T * U_dash )


        #load precomputed parameter

        SV_dash = S @ V_dash

        U_d_t_U_d_V_d = U_dash.T @ U_dash @ V_dash

        U_d_t_X_d = U_dash.T @ X_dash


        #update V_dash

        for i in range(np.size(V_dash, 0)): #i column
            for j in range(np.size(V_dash, 1)): #j column
                
                # compute the result
                result = V_dash[i,j] - ( (V_dash[i,j] * U_d_t_U_d_V_d[i,j]) / SV_dash[i,j] ) + (  (V_dash[i,j] * U_d_t_X_d[i,j] ) / SV_dash[i,j] )
                #nmf update
                V_dash[i,j] = max(0, result)

        

        #update actual V and E
        V = 




        






In [22]:
print( (-1) * np.identity(3))


X = np.random.random((3,3))
print(X, X.sum(), X.shape)

Y = np.random.random(X.shape)

r = np.subtract(X, Y)

abs_r = np.absolute(r)

#print(Y, Y.sum(), Y.shape)

print(r, r.sum(), r.shape)
print(abs_r + abs_r, abs_r.shape)
print(abs_r * 2)


c = np.concatenate((X,Y,r, np.zeros((1, X.shape[1]))), axis=0)
c = c * -1
print(c, c.shape)

print(X)


[[-1. -0. -0.]
 [-0. -1. -0.]
 [-0. -0. -1.]]
[[0.49090393 0.84779722 0.39100734]
 [0.07075819 0.94946741 0.48434641]
 [0.53524766 0.84651753 0.75217207]] 5.368217763828696 (3, 3)
[[ 0.41300388  0.73178162 -0.13618184]
 [-0.86348801  0.26325896 -0.32057392]
 [ 0.21766517  0.24818092  0.62137394]] 1.1750207226151268 (3, 3)
[[0.82600776 1.46356325 0.27236367]
 [1.72697601 0.52651791 0.64114784]
 [0.43533034 0.49636184 1.24274787]] (3, 3)
[[0.82600776 1.46356325 0.27236367]
 [1.72697601 0.52651791 0.64114784]
 [0.43533034 0.49636184 1.24274787]]
[[-0.49090393 -0.84779722 -0.39100734]
 [-0.07075819 -0.94946741 -0.48434641]
 [-0.53524766 -0.84651753 -0.75217207]
 [-0.07790005 -0.1160156  -0.52718917]
 [-0.9342462  -0.68620846 -0.80492033]
 [-0.31758249 -0.59833661 -0.13079814]
 [-0.41300388 -0.73178162  0.13618184]
 [ 0.86348801 -0.26325896  0.32057392]
 [-0.21766517 -0.24818092 -0.62137394]
 [-0.         -0.         -0.        ]] (10, 3)
[[0.49090393 0.84779722 0.39100734]
 [0.07075819 0.9